# Demography
Пример отчета по расчету социально-демографической структуры аудитории интернет-ресурсов.

## Описание задачи

Посчитаем аудиторию ресурса **Afisha** с разбивкой по социально-демографической переменной:
- Род занятий

Общие параметры:
- Период: Июль 2021, Сентябрь 2021
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Список статистик для расчета:
- Reach (reach)
- Reach% (reachPer)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Необходимо сформировать следующее задание для API Cross Web:
- расчет аудитории по ресурсу Afisha с разбивкой по соц.-дему

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-07-01', '2021-07-31'),
               ('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID ресурса
Для построения отчета необходимо получить идентификатор ресурса __Afisha__.

Для этого воспользуемся методом поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Afisha**

In [ ]:
cats.get_resource(resource='Afisha')

Таким образом, необходимый идентификатор следующий:

- **Afisha**  resourceId = 1024

### Получим название социально-демографической переменной
Для указания разбивки в задании необходимо найти название для API Cross WEB переменной **"Род занятий"**.

Для этого воспользуемся методом поиска в справочнике. Работа со справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

In [ ]:
cats.find_property('Род занятий',  expand=False)

Получили:

- Род занятий = occupation

## Задание

Перейдем к формированию задания.


### Расчет аудитории ресурса Afisha с разбивкой по соц.-дему

In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'Afisha'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID ресурса Afisha
mart_filter = 'crossMediaResourceId = 1024'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId", "occupation"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_afisha = mtask.result2table(mtask.get_result(task_audience), project_name)
df_afisha

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-afisha'))
df_info = mtask.get_report_info()
df_afisha.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()